# HW2


Before submitting your **HTpy-HW2.ipynb** file, perform the following
* `Kernel>Restart & Run All`
* Make sure that there are no errors

The following includes problems that need coding and others that only need to perform simple operations (python as a calculator).

In [ ]:
from Libraries import thermodynamics as thermo
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as csts

## 1 Insulation

This problem is an expansion of PB 3 in `HTpy-HW2.ipynb`

Assuming that the outer surface temperature of the pipe is $150^\circ C$, find an insulation that works (i.e. an insulation that will not melt, for which you can find the price and thermal conductivity online). Derive the necessary equations that allow for determination of the different costs as a function of the thickness. Generate a plot for at least 3 different thicknesses.


Hint: Derive the heat loss for a variable thickness of insulation, then apply this formula for a range of thicknesses. Compare savings of gas vs cost of insulation.

### Solution
Assumptions:
<ul>
    <li>Heat transfer is steady and 1D</li>
</ul>



In [ ]:
#Dimensional constants
L = 25. #m
#Equal spaced thickness of 0.0127 because the values for this insulation come in 1/2" thicknesses
thickness = np.arange(0.0127, 0.075, 0.0127)
rins = 0.05 #m
rout = thickness + rins #m
Aout = np.pi*2*rout*L #m^2
#Thermal constants
h = 10. #W/m^2C
T_infty = 25. #C
T_s = 150. #C
k = 0.23  #W/m*C
eps = 0.8
#Cost constants
eta_f = 0.9 
ECost = 0.02 #$/MJ
InsuCost = 17.8 #$/m at 0.0127m thickness per layer

#Thermal resitance
Rinso = np.log(rout/rins)/(2*np.pi*k*L)
Rconv = 1/(h*Aout)
Req = Rinso + Rconv
Q = (T_s-T_infty)/Req
#Cost
EnergyTotal = Q*(365*24*3600)/0.9*ECost*1e-6
InsuTotal = np.ceil(thickness/0.0127)*L*InsuCost
CostTotal = EnergyTotal + InsuTotal

#plot
plt.plot(thickness,CostTotal, '-o')
plt.xlabel('Thickness (m)')
plt.ylabel('Cost per year ($)')
plt.title('Thickness v. Cost Comparison')
for i in range(0,len(CostTotal)):
    print('The cost of operation for a year is $%.f with an insulation thickness of %.4fm.' %(CostTotal[i], thickness[i]))


## 2 Window

Set up equations and apply realistic numerical values to them to discuss heat losses of a single pane window, a single pane window with storm window and a double paned window with air trapped at a vacuum of $10^{-3} \mathrm{torr}$ ina gap of $5\mathrm{mm}$. Do not consider the effects of radiation for any of the window.

<img src="ThermalCondAir.png" alt="my awesome sketch" width=100% >

### Solution
Assumptions:
<ul>
    <li>Heat transfer is steady and 1D</li>
</ul>

In [ ]:
from scipy.optimize import fsolve

L_pane = 0.6          #m
A_pane = L_pane**2    #m^2
W_pane = 0.005        #m
L_storm = 0.6         #m
A_storm = L_storm**2  #m^2
W_storm = 0.008       #m
W_gap = 0.005         #m
T_inf_O = -10         #C
T_inf_I = 20          #C
h_O = 10              #W/m^2*C
h_I = 40              #W/m^2*C
h_B = 25              #W/m^2*C
k_air = 0.024         #W/m*C
k_glass = 0.96        #W/m*C
R_conv_O = 1/(h_O*A_pane)
R_cond_W = W_pane/(k_glass*A_pane)
R_cond_S = W_storm/(k_glass*A_storm)
R_conv_B = 1/(h_B*A_storm)
R_conv_I = 1/(h_I*A_pane)
R_conv_V = W_gap/(A_pane*k_air)
R_tot_single = R_conv_I + R_cond_W + R_conv_O
R_tot_storm = R_conv_I + R_cond_W + R_conv_B + R_cond_S + R_conv_O
R_tot_double = R_conv_I + R_cond_W + R_conv_V + R_cond_W + R_conv_O

def SinglePane(Q_loss):
    global T_inf_I, T_inf_I, R_tot_single
    return Q_loss - (T_inf_I-T_inf_O)/R_tot_single
root = fsolve(func=SinglePane, x0=[100.])
Q_L_P = root[0]

def StormWindow(Q_loss):
    global T_inf_I, T_inf_I, R_tot_storm
    return Q_loss - (T_inf_I-T_inf_O)/R_tot_storm
root = fsolve(func=StormWindow, x0=[100.])
Q_L_S = root[0]

def DoublePane(Q_loss):
    global T_inf_I, T_inf_I, R_tot_double
    return Q_loss - (T_inf_I-T_inf_O)/R_tot_double
root = fsolve(func=DoublePane, x0=[100.])
Q_L_D = root[0]

print('Heat loss from a single pane window is %.0f W' %Q_L_P)
print('Heat loss from a single pane and storm window is %.0f W' %Q_L_S)
print('Heat loss from a double pane window is %.f W' %Q_L_D)

